In [1]:

import uesgraphs as ug

from uesgraphs.examples import e1_example_readme as e1

import os
import sys
from datetime import datetime
import tempfile

In [2]:
import uesgraphs.analysis as analysis


In [16]:

workspace = e1.workspace_example("e14")

dir_ues = os.path.dirname(os.path.dirname(workspace))
pinola_json = os.path.join(dir_ues, "workspace", "e11", "inputs","test_modelgen", "Pinola", "nodes.json")
pinola_sim_data = os.path.join(dir_ues,"uesgraphs","data","Pinola_low_temp_network_inputs.mat")
pinola_sim_data = r"E:\rka_lko\work\2025_04_analysis\10042025SeestadtNewSim\Sim20250409_190504_detailed\Sim20250409_190504_1\Results\Sim20250409_190504_1_inputs.gzip"
pinola_json =r"E:\rka_lko\git\transurban_seestadt\dhc_model\workspace\transurban_seestadt_uesgraphs.json"
sysm_model = r"E:\rka_lko\git\transurban_seestadt\dhc_model\workspace\model.json"
if not os.path.exists(pinola_json):
    raise FileNotFoundError(f"File {pinola_json} not found."
                            "Please run example e11 to generate network topology.")


In [6]:

graph = ug.UESGraph()
graph.from_json(path = pinola_json, network_type="heating")
graph.graph["name"] = "pinola"
graph.graph["supply_type"] = "supply"

start_date=datetime(2024, 1, 1) 
end_date=datetime(2024, 1, 7)
    


read nodes...
******
 input_ids were {'buildings': None, 'nodes': 'f607cb63-8aed-446c-9970-9cd8380434f8', 'pipes': None, 'supplies': None}
...finished


In [5]:
gr = analysis.assign_simulation_data(graph, pinola_sim_data, start_date, end_date,aixlib_version="2.0.0",auto_retry=True)

Logfile findable here: C:\Users\rka-lko\AppData\Local\Temp\uesgraphs.analysis.data_handling.data_handling.assign_data_to_uesgraphs_20250605_181236.log


NameError: name 'get_node_values' is not defined

In [7]:
from uesgraphs.systemmodels import utilities as ut
sysm_graph = ut.load_system_model_from_json(sysm_model)

Logfile findable here: C:\Users\rka-lko\AppData\Local\Temp\3\SystemModelHeating_20250821_152127.log
Model loaded from E:\rka_lko\git\transurban_seestadt\dhc_model\workspace\model.json


In [20]:
from uesgraphs.analysis.data_handling import graph_transformation
port_mapping = graph_transformation.map_system_model_to_uesgraph(sysm_graph,graph)

In [19]:
# 1. Supply Type und Masks laden
supply_type = graph.graph["supply_type"]  # "supply" oder "return"
supply_type_prefix = {"supply": "", "return": "R"}

from uesgraphs.analysis.data_handling.data_handling import get_MASKS
masks = get_MASKS("2.1.0")  # oder deine gewünschte Version

# 2. Filter-Liste bauen (wie im existierenden Code)
filter_list = []
for edge in graph.edges:
    pipe_code = graph.edges[edge]["name"]
    for mask_type, mask_pattern in masks.items():
        var_name = mask_pattern.format(
            pipe_code=pipe_code, 
            type=supply_type_prefix[supply_type]
        )
        filter_list.append(var_name)

print(f"Filter-Liste mit {len(filter_list)} Variablen:")
for var in filter_list[:10]:  # Erste 10 zeigen
    print(f"  - {var}")

# 3. Daten laden
from uesgraphs.analysis.data_handling.data_handling import process_simulation_result
df = process_simulation_result(file_path=pinola_sim_data, filter_list=filter_list)

print(f"\nDataFrame Shape: {df.shape}")
print(f"Spalten: {list(df.columns)}")

uesgraphs.analysis.data_handling.data_handling.get_MASKS - INFO - Using AixLib version 2.1.0 masks
uesgraphs.analysis.data_handling.data_handling.process_parquet_file - INFO - Found existing gzip file: E:\rka_lko\work\2025_04_analysis\10042025SeestadtNewSim\Sim20250409_190504_detailed\Sim20250409_190504_1\Results\Sim20250409_190504_1_inputs.gzip
uesgraphs.analysis.data_handling.data_handling.process_parquet_file - INFO - Validating 72 required columns in: E:\rka_lko\work\2025_04_analysis\10042025SeestadtNewSim\Sim20250409_190504_detailed\Sim20250409_190504_1\Results\Sim20250409_190504_1_inputs.gzip


Filter-Liste mit 72 Variablen:
  - networkModel.pipe10021010.port_a.m_flow
  - networkModel.pipe10021010.dp
  - networkModel.pipe10021010.port_a.p
  - networkModel.pipe10021010.port_b.p
  - networkModel.pipe10021010.sta_a.T
  - networkModel.pipe10021010.sta_b.T
  - networkModel.pipe10021014.port_a.m_flow
  - networkModel.pipe10021014.dp
  - networkModel.pipe10021014.port_a.p
  - networkModel.pipe10021014.port_b.p


uesgraphs.analysis.data_handling.data_handling.process_parquet_file - INFO - ✅ All required columns found in data file
uesgraphs.analysis.data_handling.data_handling.process_parquet_file - INFO - Starting parquet file processing: E:\rka_lko\work\2025_04_analysis\10042025SeestadtNewSim\Sim20250409_190504_detailed\Sim20250409_190504_1\Results\Sim20250409_190504_1_inputs.gzip
uesgraphs.analysis.data_handling.data_handling.process_parquet_file - INFO - Successfully loaded 35041 rows, 72 columns



DataFrame Shape: (35041, 72)
Spalten: ['networkModel.pipe10021010.port_a.m_flow', 'networkModel.pipe10021010.dp', 'networkModel.pipe10021010.port_a.p', 'networkModel.pipe10021010.port_b.p', 'networkModel.pipe10021010.sta_a.T', 'networkModel.pipe10021010.sta_b.T', 'networkModel.pipe10021014.port_a.m_flow', 'networkModel.pipe10021014.dp', 'networkModel.pipe10021014.port_a.p', 'networkModel.pipe10021014.port_b.p', 'networkModel.pipe10021014.sta_a.T', 'networkModel.pipe10021014.sta_b.T', 'networkModel.pipe10021025.port_a.m_flow', 'networkModel.pipe10021025.dp', 'networkModel.pipe10021025.port_a.p', 'networkModel.pipe10021025.port_b.p', 'networkModel.pipe10021025.sta_a.T', 'networkModel.pipe10021025.sta_b.T', 'networkModel.pipe10041006.port_a.m_flow', 'networkModel.pipe10041006.dp', 'networkModel.pipe10041006.port_a.p', 'networkModel.pipe10041006.port_b.p', 'networkModel.pipe10041006.sta_a.T', 'networkModel.pipe10041006.sta_b.T', 'networkModel.pipe10041007.port_a.m_flow', 'networkModel.pip

In [39]:
masks

{'m_flow': 'networkModel.pipe{pipe_code}{type}.port_a.m_flow',
 'dp': 'networkModel.pipe{pipe_code}{type}.dp',
 'p_a': 'networkModel.pipe{pipe_code}{type}.port_a.p',
 'p_b': 'networkModel.pipe{pipe_code}{type}.port_b.p',
 'T_a': 'networkModel.pipe{pipe_code}{type}.sta_a.T',
 'T_b': 'networkModel.pipe{pipe_code}{type}.sta_b.T'}

In [40]:
port

'pipe10021014.port_a'

In [47]:
i = 1001
node_ports = port_mapping[i]  # Liste aller Ports für diesen Node
collected_values = {"T": [], "p": []}

# Mapping für flexible Attributnamen im Graph
attribute_names = {"T": "temperature", "p": "press_flow"}  # Konfigurierbar!

for port in node_ports:  # Iteriere über ALLE Ports des Nodes
    port_splitted = port.split(".")
    if port_splitted[1].endswith("a"):
        #port is an a port
        temp = f"networkModel.{port_splitted[0]}.sta_a.T"
        press = f"networkModel.{port_splitted[0]}.port_a.p"
        t_value = df[temp].iloc[0]
        p_value = df[press].iloc[0]
        collected_values["T"].append(t_value)
        collected_values["p"].append(p_value)

# Validiere und weise zu
for size in ["T", "p"]:
    unique_vals = set(collected_values[size])
    if len(unique_vals) != 1:
        raise ValueError(f"Node {i}: Inkonsistente {size}-Werte gefunden: {collected_values[size]}")
    graph.nodes[i][attribute_names[size]] = collected_values[size][0]

In [ ]:
print(f"Port Mapping für Knoten {i}: {port}")
pipe_mask = df.columns.str.contains('pipe10021014.')
df_pipe = df.loc[:, pipe_mask]
print(f"Gefundene Spalten: {df.columns[pipe_mask].tolist()}")

In [ ]:
edges = list(graph.edges)
for edge in edges:
    print(f"Edge {edge} has data: {graph.edges[edge]}")

Edge (1001, 1005) has data: {'diameter': 0.0697, 'length': 62.67448664094058, 'pipeID': '10021010', 'name': '10021010', 'node_0': 1002, 'node_1': 1010}
Edge (1001, 1006) has data: {'diameter': 0.2101, 'length': 40.10060652984215, 'pipeID': '10021014', 'name': '10021014', 'node_0': 1002, 'node_1': 1014}
Edge (1001, 1009) has data: {'diameter': 0.2101, 'length': 23.793988913832845, 'pipeID': '10021025', 'name': '10021025', 'node_0': 1002, 'node_1': 'supply1'}
Edge (1002, 1003) has data: {'diameter': 0.1325, 'length': 23.15227945754485, 'pipeID': '10041006', 'name': '10041006', 'node_0': 1004, 'node_1': 1006}
Edge (1002, 1004) has data: {'diameter': 0.1325, 'length': 67.21343661070965, 'pipeID': '10041007', 'name': '10041007', 'node_0': 1004, 'node_1': 1007}
Edge (1002, 1011) has data: {'diameter': 0.0539, 'length': 25.20026053450003, 'pipeID': '10041027', 'name': '10041027', 'node_0': 1004, 'node_1': 'w11_2'}
Edge (1003, 1008) has data: {'diameter': 0.2101, 'length': 43.10762795752977, '

In [ ]:
    graph = analyze.assign_data_to_uesgraphs(graph,sim_data = pinola_sim_data,
                                             start_date=start_date,
                                             end_date=end_date,
                                             aixlib_version="2.1.0") #aixlib version is needed to assign data properly
    
    ### Plotting
    #Visuals cant handle series data, so we just take the mean values, but single time points are possible
    vis = ug.Visuals(graph)
    for edge in graph.edges:
        graph.edges[edge]["m_flow_mean"] = graph.edges[edge]["m_flow"].mean()
    vis.show_network(show_plot=False,
                           scaling_factor=1,
                           scaling_factor_diameter=50,
                           label_size=15,
                           ylabel="Mean mass flow [kg/s]",
                           generic_extensive_size="m_flow_mean",
                           save_as=os.path.join(workspace, "m_flow.png"),
                           timestamp=f"{graph.graph["name"]}: Mean mass flow"
                           )
    
    for node in graph.nodes:
        graph.nodes[node]["press_flow_mean"] = graph.nodes[node]["press_flow"].mean()
    vis.show_network(show_plot=False,
                           scaling_factor=1,
                           scaling_factor_diameter=50,
                           ylabel="Mean pressure [Pa]",
                           label_size=15,
                           generic_intensive_size="press_flow_mean",
                           save_as=os.path.join(workspace, "press_flow.png"),
                           timestamp=f"{graph.graph["name"]}: Mean pressure flow"
                           )
    
    df = analyze.pump_power_analysis(graph, True, r"D:\rka-lko\work\2025_04_analysis\3")

if __name__ == "__main__":
    main()

In [ ]:
# Example: Before/After demonstration of node value assignment
print("=== BEFORE NODE VALUE ASSIGNMENT ===")
sample_nodes = list(graph.nodes)[:5]  # Take first 5 nodes as examples

print("Sample nodes before assignment:")
for node_id in sample_nodes:
    node_data = graph.nodes[node_id]
    temp = node_data.get('temperature', 'NOT SET')
    pressure = node_data.get('pressure', 'NOT SET')
    print(f"Node {node_id}: Temperature = {temp}, Pressure = {pressure}")

print(f"\nTotal nodes in graph: {len(graph.nodes)}")
nodes_with_temp = sum(1 for n in graph.nodes if 'temperature' in graph.nodes[n])
nodes_with_pressure = sum(1 for n in graph.nodes if 'pressure' in graph.nodes[n])
print(f"Nodes with temperature: {nodes_with_temp}")
print(f"Nodes with pressure: {nodes_with_pressure}")

# Run the assignment
print("\n" + "="*50)
print("RUNNING NODE VALUE ASSIGNMENT...")
print("="*50)

config = {
    "temperature": ["sta_{port}.T"],
    "pressure": ["port_{port}.p"], 
}

processed, inconsistencies, conflicts = assign_node_values_flexible(graph, df, port_mapping, config)

print("\n=== AFTER NODE VALUE ASSIGNMENT ===")
print("Sample nodes after assignment:")
for node_id in sample_nodes:
    node_data = graph.nodes[node_id]
    temp = node_data.get('temperature', 'STILL NOT SET')
    pressure = node_data.get('pressure', 'STILL NOT SET')
    if isinstance(temp, float):
        temp = f"{temp:.3f}"
    if isinstance(pressure, float):
        pressure = f"{pressure:.3f}"
    print(f"Node {node_id}: Temperature = {temp}, Pressure = {pressure}")

# Summary statistics
nodes_with_temp_after = sum(1 for n in graph.nodes if 'temperature' in graph.nodes[n])
nodes_with_pressure_after = sum(1 for n in graph.nodes if 'pressure' in graph.nodes[n])
print(f"\nSUMMARY:")
print(f"Nodes with temperature: {nodes_with_temp} → {nodes_with_temp_after}")
print(f"Nodes with pressure: {nodes_with_pressure} → {nodes_with_pressure_after}")
print(f"Success rate: {nodes_with_temp_after}/{len(graph.nodes)} = {nodes_with_temp_after/len(graph.nodes)*100:.1f}%")

In [ ]:
def assign_node_values_flexible(graph, df, port_mapping, attribute_config, time_index=0):
    """
    Flexible node value assignment with pattern validation
    
    Now checks if multiple patterns for the same attribute give consistent results
    """
    
    processed_count = 0
    inconsistency_count = 0
    pattern_conflicts = 0
    
    for node_id, node_ports in port_mapping.items():
        if not node_ports:
            continue
            
        for attr_name, patterns in attribute_config.items():
            all_values_by_pattern = {}  # Track values from each pattern
            
            for port in node_ports:
                port_parts = port.split(".")
                pipe_name = port_parts[0]
                port_suffix = port_parts[1].split("_")[-1]  # Extract 'a' or 'b'
                
                # Test ALL patterns and collect their values
                for pattern in patterns:
                    col_name = f"networkModel.{pipe_name}.{pattern.format(port=port_suffix)}"
                    if col_name in df.columns:
                        value = df[col_name].iloc[time_index]
                        
                        if pattern not in all_values_by_pattern:
                            all_values_by_pattern[pattern] = []
                        all_values_by_pattern[pattern].append(value)
            
            # Now check if different patterns give consistent results
            if len(all_values_by_pattern) > 1:
                # Multiple patterns found data - check consistency
                pattern_means = {}
                for pattern, values in all_values_by_pattern.items():
                    pattern_means[pattern] = sum(values) / len(values)
                
                # Check if all patterns agree (within tolerance)
                mean_values = list(pattern_means.values())
                if len(set(round(v, 6) for v in mean_values)) > 1:
                    print(f"Pattern conflict for {attr_name} at node {node_id}:")
                    for pattern, mean_val in pattern_means.items():
                        print(f"  {pattern}: {mean_val:.6f}")
                    pattern_conflicts += 1
            
            # Use values from the first successful pattern
            if all_values_by_pattern:
                first_pattern = list(all_values_by_pattern.keys())[0]
                collected_values = all_values_by_pattern[first_pattern]
                
                # Check consistency within the chosen pattern
                unique_vals = set(round(v, 6) for v in collected_values)
                
                if len(unique_vals) == 1:
                    graph.nodes[node_id][attr_name] = collected_values[0]
                else:
                    mean_val = sum(collected_values) / len(collected_values)
                    graph.nodes[node_id][attr_name] = mean_val
                    print(f"Warning: Node {node_id} inconsistent {attr_name} within pattern: using mean {mean_val:.3f}")
                    inconsistency_count += 1
        
        processed_count += 1
    
    print(f"Processed {processed_count} nodes")
    print(f"Within-pattern inconsistencies: {inconsistency_count}")
    print(f"Cross-pattern conflicts: {pattern_conflicts}")
    return processed_count, inconsistency_count, pattern_conflicts

# Usage example with configurable patterns
config = {
    "temperature": ["sta_{port}.T"],
    "pressure": ["port_{port}.p"], 
    # Future example: test multiple patterns for enthalpy
    # "enthalpy": ["port_{port}.h", "sta_{port}.h"]  
}

processed, inconsistencies, conflicts = assign_node_values_flexible(graph, df, port_mapping, config)